In [1]:
import pandas as pd
from sqlalchemy import create_engine


In [2]:
DB_USER = "root"
DB_PASSWORD = "1433"
DB_HOST = "localhost"
DB_NAME = "financial_analytics"

engine = create_engine(
    f"mysql+mysqlconnector://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}"
)


In [3]:
query = """
SELECT asset_id, trade_date, adj_close, volume
FROM market_prices
"""
df = pd.read_sql(query, engine)

df.head()


,asset_id,trade_date,adj_close,volume
0,11,2019-01-02,2510.030029,3733160000
1,11,2019-01-03,2447.889893,3858830000
2,11,2019-01-04,2531.939941,4234140000
3,11,2019-01-07,2549.689941,4133120000
4,11,2019-01-08,2574.409912,4120060000


In [4]:
df.shape


(19503, 4)

In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19503 entries, 0 to 19502
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   asset_id    19503 non-null  int64  
 1   trade_date  19503 non-null  object 
 2   adj_close   19503 non-null  float64
 3   volume      19503 non-null  int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 609.6+ KB


In [6]:
df.isnull().sum()


asset_id      0
trade_date    0
adj_close     0
volume        0
dtype: int64

In [7]:
df.duplicated(subset=["asset_id", "trade_date"]).sum()


np.int64(0)

In [8]:
df.groupby(["asset_id", "trade_date"]).size().value_counts()


1    19503
Name: count, dtype: int64

In [9]:
df.groupby("asset_id")["trade_date"].agg(["min", "max"])


,min,max
asset_id,,
1,2019-01-02,2026-01-21
2,2019-01-02,2026-01-21
3,2019-01-02,2026-01-21
4,2019-01-02,2026-01-21
5,2019-01-02,2026-01-21
6,2019-01-02,2026-01-21
7,2019-01-02,2026-01-21
8,2019-01-02,2026-01-21
9,2019-01-02,2026-01-21


In [10]:
df["pct_change"] = df.groupby("asset_id")["adj_close"].pct_change()

df[df["pct_change"].abs() > 0.5]


,asset_id,trade_date,adj_close,volume,pct_change


In [11]:
clean_df = df.drop(columns=["pct_change"], errors="ignore")

clean_df.to_csv(
    "../data/processed/clean_market_prices_week1.csv",
    index=False
)
